In [255]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
plt.style.use('./deeplearning.mplstyle')
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense
import logging
logging.getLogger("tensorflow").setLevel(logging.ERROR)
tf.autograph.set_verbosity(0)

RANDOM_STATE = 42

In [ ]:
# Load the dataset using pandas
df = pd.read_csv("data.csv")

print(df.head())
df = df.drop('I', axis=1)

## Removing our target variable

selected_features = ["qPA", "Pulse", "BreathFreq"]
X = df[selected_features].values
y_gravity = df["Gravity"].values

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)


print(f"Temperature Max, Min post normalization: {np.max(X_scaled[0]):0.2f}, {np.min(X_scaled[0]):0.2f}")

   I     P_sist     P_dist       qPA       Pulse  BreathFreq    Gravity  Class
0  1  13.592433  12.220855  8.416754   75.921057   21.635259  40.000000      2
1  2  15.775386  13.586879  8.725890   63.813564   19.718734  41.530427      2
2  3   3.649369   1.904802  0.000000  197.210213   19.045471  52.730745      3
3  4  17.264362  13.700638  8.733333  143.636181   17.621141  34.679911      2
4  5  12.705183   9.485389  1.747626   82.636672   12.209535  69.375882      3
Temperature Max, Min post normalization: 1.72, -0.44


In [257]:
cat_variables = ['Class']

# This will replace the columns with the one-hot encoded ones and keep the columns outside 'columns' argument as it is.
df = pd.get_dummies(data = df,
                         prefix = "class",
                         columns = cat_variables)

print(df.head())

      P_sist     P_dist       qPA       Pulse  BreathFreq    Gravity  class_1  \
0  13.592433  12.220855  8.416754   75.921057   21.635259  40.000000    False   
1  15.775386  13.586879  8.725890   63.813564   19.718734  41.530427    False   
2   3.649369   1.904802  0.000000  197.210213   19.045471  52.730745    False   
3  17.264362  13.700638  8.733333  143.636181   17.621141  34.679911    False   
4  12.705183   9.485389  1.747626   82.636672   12.209535  69.375882    False   

   class_2  class_3  class_4  
0     True    False    False  
1     True    False    False  
2    False     True    False  
3     True    False    False  
4    False     True    False  


In [269]:
# Define X (features), y_class e y_gravity
y_class = df[["class_1", "class_2", "class_3", "class_4"]].values

# Divide em treino e teste
X_train, X_test, y_train, y_test, y_class_train, y_class_test = train_test_split(X, y_gravity, y_class, train_size = 0.7, random_state = RANDOM_STATE)

print(f'train samples: {len(X_train)}\ntest samples: {len(X_test)}')

train samples: 1050
test samples: 450


In [259]:
print(X_train.shape, y_class_train.shape)
print(X.shape[1])

(1050, 3) (900, 4)
3


In [260]:
tf.random.set_seed(1234)  # applied to achieve consistent results

model = tf.keras.Sequential([
    tf.keras.Input(X.shape[1]),
    Dense(32, activation='relu', name = 'layer1'),
    Dense(16, activation='relu', name = 'layer2'),
    Dense(8, activation='relu', name = 'layer3'),
    Dense(1, activation='linear', name = 'output')  # Output for regression
])


In [261]:
model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 layer1 (Dense)              (None, 32)                128       
                                                                 
 layer2 (Dense)              (None, 16)                528       
                                                                 
 layer3 (Dense)              (None, 8)                 136       
                                                                 
 output (Dense)              (None, 1)                 9         
                                                                 
Total params: 801
Trainable params: 801
Non-trainable params: 0
_________________________________________________________________


In [262]:
model.compile(
    optimizer='adam',
    loss='mean_squared_error',
    metrics=['mae']
)
early_stop = EarlyStopping(
    monitor='val_loss',        # what metric to monitor (can also use 'val_mae')
    patience=10,               # how many epochs to wait before stopping
    restore_best_weights=True # keep the best model, not the last one
)
model.fit(
    X_train, y_train,
    epochs=4000,
    callbacks=[early_stop],    # here's the EarlyStopping callback
    verbose=1
)

Epoch 1/4000
33/33 [==============================] - 0s 1ms/step - loss: 1108.3315 - mae: 28.7492
Epoch 2/4000
33/33 [==============================] - 0s 1ms/step - loss: 669.3345 - mae: 22.1337
Epoch 3/4000
33/33 [==============================] - 0s 2ms/step - loss: 599.0907 - mae: 20.8381
Epoch 4/4000
33/33 [==============================] - 0s 2ms/step - loss: 519.1530 - mae: 19.1873
Epoch 5/4000
33/33 [==============================] - 0s 2ms/step - loss: 418.4390 - mae: 17.0200
Epoch 6/4000
33/33 [==============================] - 0s 2ms/step - loss: 376.8008 - mae: 15.8936
Epoch 7/4000
33/33 [==============================] - 0s 2ms/step - loss: 350.2897 - mae: 15.1347
Epoch 8/4000
33/33 [==============================] - 0s 2ms/step - loss: 353.7630 - mae: 15.1006
Epoch 9/4000
33/33 [==============================] - 0s 2ms/step - loss: 341.4106 - mae: 14.7438
Epoch 10/4000
33/33 [==============================] - 0s 2ms/step - loss: 334.9610 - mae: 14.5900
Epoch 11/4000
33/3

In [266]:
def regression_accuracy(model, X_test, y_test, tolerance=3.7):
    """
    Calculates the percentage of predictions within a tolerance of the true value.
    Args:
        model: Trained Keras model
        X_test: Test features
        y_test: True values
        tolerance: Acceptable error (absolute difference)
    Returns:
        accuracy: Percentage of predictions within tolerance
    """
    y_pred = model.predict(X_test).flatten()
    correct = np.abs(y_pred - y_test) <= tolerance
    accuracy = np.mean(correct)
    print(f"Regression accuracy (within ±{tolerance}): {accuracy*100:.2f}%")
    return accuracy

# Example usage:
regression_accuracy(model, X_test, y_test)

15/15 [==============================] - 0s 1ms/step
Regression accuracy (within ±3.7): 82.22%


0.8222222222222222

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)


15/15 [==============================] - 0s 2ms/step
(450, 1)


In [ ]:
y_pred = model.predict(X_test)  # shape: (num_samples, 1)

# Suppose you have y_class_test as integer labels (not one-hot)
num_classes = len(np.unique(y_class_test))

# Build the classifier
clf_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1,)),  # Input is y_pred from regression
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

clf_model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Train the classifier
clf_model.fit(y_pred, y_pred, epochs=50, verbose=1)

15/15 [==============================] - 0s 6ms/step
Epoch 1/50


ValueError: in user code:

    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "c:\Users\guilh\anaconda3\envs\tf\lib\site-packages\keras\engine\input_spec.py", line 277, in assert_input_compatibility
    

In [ ]:
y_class_pred = np.argmax(clf_model.predict(y_pred), axis=1)
print("Predicted classes:", y_class_pred)